from gensim.models import Word2Vec
from konlpy.tag import Okt  # 한국어 처리를 위한 라이브러리
import multiprocessing
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from sklearn.metrics.pairwise import cosine_similarity

In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [2]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt


In [3]:
import torch

torch.__version__

'2.1.1'

In [4]:
import torch

# PyTorch 버전 확인
print("PyTorch 버전:", torch.__version__)

# CUDA 사용 가능 여부 및 버전 확인
if torch.cuda.is_available():
    print("CUDA 사용 가능")
    print("CUDA 버전:", torch.version.cuda)
else:
    print("CUDA 사용 불가")

PyTorch 버전: 2.1.1
CUDA 사용 가능
CUDA 버전: 12.1


In [5]:
df=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/df.csv',encoding='cp949')
df

,화물키,화물정보,상차지,상차경도,상차위도,하차지,하차경도,하차위도,차종류,차톤수,...,직선거리,등록시간,배차시간,화물실중량,등록날짜,요일,차주키,운행시간,잡힌시간,카테고리
0,92218976,cj택배 셔틀,경기 광주 도척면,127.319476,37.309033,대전 대덕 문평동,127.401436,36.446504,윙바디,11톤,...,116,2020-04-01 0:03,2020-04-01 0:09,7,2020-04-01 00:00:00+00:00,Wednesday,47982,0 days 06:00:00,0 days 00:06:00,택배
1,92218977,cj택배 셔틀,경기 광주 도척면,127.319476,37.309033,충북 청원,127.532134,36.557306,윙바디,11톤,...,113,2020-04-01 0:04,2020-04-01 0:07,7,2020-04-01 00:00:00+00:00,Wednesday,85445,0 days 06:00:00,0 days 00:03:00,택배
2,107700730,쿠팡 1 입차 2시 출차 가는대로 하차,경북 칠곡 지천면,128.478897,35.966001,충남 천안 동남구 수신면,127.285123,36.724336,윙바디,11톤,...,178,2020-04-01 0:15,2020-04-01 0:15,5,2020-04-01 00:00:00+00:00,Wednesday,83154,0 days 03:00:00,0 days 00:00:00,택배
3,107700731,지대17파렛 가는대로상차,충북 제천 ic옆,128.190948,37.132582,충남 예산 신암면,126.826259,36.670637,카고윙바디,25톤,...,165,2020-04-01 0:18,2020-04-01 0:19,27,2020-04-01 00:00:00+00:00,Wednesday,54972,0 days 05:00:00,0 days 00:01:00,파렛트
4,107700732,수지백,경북 영천 도남동,128.935892,35.930911,충남 아산 인주면,126.884683,36.868970,카고윙바디,25톤,...,250,2020-04-01 0:22,2020-04-01 3:20,22,2020-04-01 00:00:00+00:00,Wednesday,51336,0 days 19:00:00,0 days 02:58:00,수지
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903193,1216822,예약당착냉연코일1개 19,충남 당진 송악면,126.742749,36.924837,인천,126.705206,37.456256,카고,25톤,...,87,2020-09-29 21:19,2020-09-29 22:40,19,2020-09-01 00:00:00+00:00,Tuesday,41999,0 days 07:00:00,0 days 01:21:00,철재
903194,1216750,생수18p 상하차빠름 내일착 인수증즉시결,경남 산청 시천면,127.840601,35.278409,경기 시흥,126.803102,37.379888,카고윙바디,18톤,...,315,2020-09-29 21:23,2020-09-29 21:38,18,2020-09-01 00:00:00+00:00,Tuesday,91555,0 days 11:00:00,0 days 00:15:00,음료
903195,1216876,음료수 새벽6시 상차,경기 여주,127.637204,37.298458,서울 송파,127.018261,37.589071,윙바디,11톤,...,82,2020-09-29 21:38,2020-09-30 0:10,10,2020-09-01 00:00:00+00:00,Tuesday,21466,0 days 04:00:00,0 days 02:32:00,음료
903196,1216768,단조파레트,경남 밀양 부북면,128.715734,35.516034,경북 경주 내남면,129.177144,35.750409,카고윙바디,25톤,...,80,2020-09-29 21:53,2020-09-29 21:55,25,2020-09-01 00:00:00+00:00,Tuesday,50722,0 days 02:30:00,0 days 00:02:00,철재


In [6]:
df['상차시간'] = pd.to_datetime(df['상차시간'])
df['하차시간'] = pd.to_datetime(df['하차시간'])
df['등록시간'] = pd.to_datetime(df['등록시간'])
df['배차시간'] = pd.to_datetime(df['배차시간'])
df['등록날짜'] = pd.to_datetime(df['등록날짜'])
df['운행시간'] = pd.to_timedelta(df['운행시간'])
df['잡힌시간'] = pd.to_timedelta(df['잡힌시간'])
df['상차시간'] = df['상차시간'].astype(np.int64) // 10**9
df['하차시간'] = df['하차시간'].astype(np.int64) // 10**9
df['운행시간'] = df['운행시간'].astype(np.int64) // 10**9
df['잡힌시간'] = df['잡힌시간'].astype(np.int64) // 10**9
df['등록시간'] = df['등록시간'].astype(np.int64) // 10**9
df['배차시간'] = df['배차시간'].astype(np.int64) // 10**9

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903198 entries, 0 to 903197
Data columns (total 27 columns):
 #   Column  Non-Null Count   Dtype              
---  ------  --------------   -----              
 0   화물키     903198 non-null  int64              
 1   화물정보    903198 non-null  object             
 2   상차지     903198 non-null  object             
 3   상차경도    903198 non-null  float64            
 4   상차위도    903196 non-null  float64            
 5   하차지     903198 non-null  object             
 6   하차경도    903198 non-null  float64            
 7   하차위도    903198 non-null  float64            
 8   차종류     903198 non-null  object             
 9   차톤수     903198 non-null  object             
 10  상차시간    903198 non-null  int64              
 11  하차시간    903198 non-null  int64              
 12  결제방법    903198 non-null  object             
 13  기사운임    903198 non-null  int64              
 14  화물종류    903198 non-null  object             
 15  상차방법    903198 non-null  object   

In [8]:
# 인덱스를 기준으로 데이터 프레임을 분할
train_df = df[df.index <= 758353] # 4월 ~ 8월
test_df = df[df.index > 758353] # 9월

# 훈련 세트와 테스트 세트의 크기 확인
print("훈련 세트 크기:", train_df.shape)
print("테스트 세트 크기:", test_df.shape)

# 84% 대 16%

훈련 세트 크기: (758354, 27)
테스트 세트 크기: (144844, 27)


In [9]:
# # 임시 데이터 셋
# train_df = train_df[:84000]
# test_df = test_df[:16000]


In [10]:
tokenizer = Okt()

In [11]:
train_df['차주_vector_data'] = train_df['차종류'].astype(str) + train_df['차톤수'].astype(str) + train_df['기사운임'].astype(str) + train_df['결제방법'].astype(str) + train_df['잡힌시간'].astype(str) + train_df['운행시간'].astype(str) 

train_df['화물_vector_data'] = train_df['화물종류'].astype(str)+ train_df['상차방법'].astype(str) + train_df['하차방법'].astype(str) + train_df['카테고리'].astype(str) + train_df['화물실중량'].astype(str) + train_df['직선거리'].astype(str)+ train_df['상차지'].astype(str)+ train_df['하차지'].astype(str)+ train_df['요일'].astype(str)

test_df['차주_vector_data'] = test_df['차종류'].astype(str) + test_df['차톤수'].astype(str) + test_df['기사운임'].astype(str) + test_df['결제방법'].astype(str) + test_df['잡힌시간'].astype(str) + test_df['운행시간'].astype(str) 
test_df['화물_vector_data'] = test_df['화물종류'].astype(str)+ test_df['상차방법'].astype(str) + test_df['하차방법'].astype(str) + test_df['카테고리'].astype(str) + test_df['화물실중량'].astype(str) + test_df['직선거리'].astype(str)+ test_df['상차지'].astype(str)+ test_df['하차지'].astype(str)+ test_df['요일'].astype(str)

C:\Users\user\AppData\Local\Temp\ipykernel_13020\3045285219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['차주_vector_data'] = train_df['차종류'].astype(str) + train_df['차톤수'].astype(str) + train_df['기사운임'].astype(str) + train_df['결제방법'].astype(str) + train_df['잡힌시간'].astype(str) + train_df['운행시간'].astype(str)
C:\Users\user\AppData\Local\Temp\ipykernel_13020\3045285219.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['화물_vector_data'] = train_df['화물종류'].astype(str)+ train_df['상차방법'].a

In [12]:
# Train Dataset Word2Vec 모델 학습

# min_coount = 1로 설정하여 희귀한 차종류도 고려

# 차주_vector 모델
tokenized_vehicle_data = [tokenizer.morphs(text) for text in train_df['차주_vector_data']]
vehicle_model = Word2Vec(tokenized_vehicle_data, vector_size=100, window=5, min_count=1, workers=20)

# 화물_vector 모델
tokenized_cargo_data = [tokenizer.morphs(text) for text in train_df['화물_vector_data']]
cargo_model = Word2Vec(tokenized_cargo_data, vector_size=100, window=5, min_count=1, workers=20)

# 벡터 변환 함수
def text_to_vector(text, model):
    tokens = tokenizer.morphs(text)
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# 차주_vector 및 화물_vector에 대한 벡터 계산
train_df['차주_vector'] = train_df['차주_vector_data'].apply(lambda x: text_to_vector(x, vehicle_model))
train_df['화물_vector'] = train_df['화물_vector_data'].apply(lambda x: text_to_vector(x, cargo_model))

# 결과 확인
print(train_df[['차주_vector', '화물_vector']].head())

C:\Users\user\AppData\Local\Temp\ipykernel_13020\1335232851.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['차주_vector'] = train_df['차주_vector_data'].apply(lambda x: text_to_vector(x, vehicle_model))


                                           차주_vector  \
0  [0.121088564, 0.37731013, -0.5056876, -0.23791...   
1  [0.12250619, 0.39100736, -0.44513074, -0.23892...   
2  [-0.032365344, 0.3176577, -0.58045286, -0.2556...   
3  [-0.1274778, 0.43833312, -1.0016648, -0.371597...   
4  [-0.25433117, 0.3576504, -1.0893116, -0.305767...   

                                           화물_vector  
0  [0.22807461, 0.4544489, 0.37247446, -0.3553687...  
1  [0.28094462, 0.8215793, 0.68415004, -0.5493366...  
2  [-0.18613616, 1.1084411, 0.54273486, -0.444575...  
3  [0.58452743, 0.36189213, 0.97288823, -0.696359...  
4  [0.06618146, 0.5210608, 0.9576342, 0.11015105,...  


C:\Users\user\AppData\Local\Temp\ipykernel_13020\1335232851.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['화물_vector'] = train_df['화물_vector_data'].apply(lambda x: text_to_vector(x, cargo_model))


In [13]:
# Test Dataset Word2Vec 모델 학습

# min_coount = 1로 설정하여 희귀한 차종류도 고려

# 차주_vector 모델
tokenized_vehicle_data = [tokenizer.morphs(text) for text in test_df['차주_vector_data']]
vehicle_model = Word2Vec(tokenized_vehicle_data, vector_size=100, window=5, min_count=1, workers=20)

# 화물_vector 모델
tokenized_cargo_data = [tokenizer.morphs(text) for text in test_df['화물_vector_data']]
cargo_model = Word2Vec(tokenized_cargo_data, vector_size=100, window=5, min_count=1, workers=20)

# 벡터 변환 함수
def text_to_vector(text, model):
    tokens = tokenizer.morphs(text)
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# 차주_vector 및 화물_vector에 대한 벡터 계산
test_df['차주_vector'] = test_df['차주_vector_data'].apply(lambda x: text_to_vector(x, vehicle_model))
test_df['화물_vector'] = test_df['화물_vector_data'].apply(lambda x: text_to_vector(x, cargo_model))

# 결과 확인
print(test_df[['차주_vector', '화물_vector']].head())

C:\Users\user\AppData\Local\Temp\ipykernel_13020\1961867442.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['차주_vector'] = test_df['차주_vector_data'].apply(lambda x: text_to_vector(x, vehicle_model))


                                                차주_vector  \
758354  [-0.780802, 0.3218931, 0.48848376, -0.28124806...   
758355  [-0.791671, 0.38675636, 0.5277456, -0.34303918...   
758356  [-0.82090235, 0.27039382, 0.53074116, -0.25630...   
758357  [-0.76828253, 0.30473742, 0.48556015, -0.27985...   
758358  [-0.74638563, 0.3401409, 0.4229104, -0.2780772...   

                                                화물_vector  
758354  [-0.7504131, 0.40451428, -0.6053661, -0.343770...  
758355  [0.217625, 0.4669302, -0.93741536, 0.2732704, ...  
758356  [-0.70601654, 0.050378554, -0.6243256, -0.0437...  
758357  [-0.2711099, 0.107471906, -0.7312391, -0.13891...  
758358  [-0.685044, 0.0040595606, -0.818091, -0.209053...  


C:\Users\user\AppData\Local\Temp\ipykernel_13020\1961867442.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['화물_vector'] = test_df['화물_vector_data'].apply(lambda x: text_to_vector(x, cargo_model))


In [21]:
def recommend_cargos_within_test_df(test_df, top_n=5):
    recommendations_list = []

    # test_df에서 화물 벡터 준비
    cargo_vectors = np.array(list(test_df['화물_vector']))
    cargo_keys = test_df['화물키'].values
    
    # test_df를 차주키 기준으로 중복 없이 순회
    for driver_key in test_df['차주키'].unique():
        # 현재 차주의 벡터 가져오기
        driver_vector = np.array(test_df[test_df['차주키'] == driver_key]['차주_vector'].iloc[0]).reshape(1, -1)
        
        # 코사인 유사도 계산
        similarity_scores = cosine_similarity(driver_vector, cargo_vectors)[0]
        
        # 가장 유사도가 높은 화물 인덱스 찾기 (상위 n개)
        top_cargo_indices = np.argsort(similarity_scores)[-top_n:][::-1]
        
        # 상위 n개 화물의 키와 코사인 유사도 추출
        for rank, cargo_index in enumerate(top_cargo_indices):
            recommended_cargo_key = cargo_keys[cargo_index]
            similarity_score = similarity_scores[cargo_index]
            
            recommendations_list.append({
                '차주키': driver_key,
                '추천_화물키': recommended_cargo_key,
                '코사인_유사도': similarity_score,
                '순위': rank + 1
            })

    return pd.DataFrame(recommendations_list)

# 추천 실행 및 결과 출력
recommendations_within_test_df = recommend_cargos_within_test_df(test_df, top_n=30)
print(recommendations_within_test_df)


          차주키  추천_화물키   코사인_유사도  순위
0       56129  947973  0.278829   1
1       56129  881213  0.274529   2
2       56129  884111  0.260324   3
3       56129  750178  0.253518   4
4       56129  884869  0.248656   5
...       ...     ...       ...  ..
459955  89302  902060  0.247680  26
459956  89302  787098  0.247632  27
459957  89302  786598  0.247632  28
459958  89302  141758  0.246338  29
459959  89302  550370  0.246325  30

[459960 rows x 4 columns]


In [24]:
recommendations_within_test_df.to_csv('1234.csv', index=False,encoding='cp949')

In [16]:
# recommendations_with_driver_info = pd.merge(recommendations_with_similarity, test_df, on='차주키', how='left')

In [17]:
# recommendations_with_cargo_info = pd.merge(recommendations_with_driver_info, train_df, left_on='추천_화물키', right_on='화물키', how='left', suffixes=('_차주', '_화물'))
# recommendations_with_cargo_info

In [18]:
# 차주키별로 화물키 리스트 생성
train_cargo_keys_per_driver = train_df.groupby('차주키')['화물키'].apply(list).reset_index()

# 화물키에 해당하는 화물_vector를 찾아 리스트로 반환하는 함수
def get_cargo_vectors(cargo_keys):
    vectors = train_df[train_df['화물키'].isin(cargo_keys)]['화물_vector'].tolist()
    return vectors

# 차주키별로 화물키 리스트 생성
train_cargo_keys_per_driver = train_df.groupby('차주키')['화물키'].apply(list).reset_index()

# 차주키별로 화물키에 해당하는 화물_vector 리스트 추가
train_cargo_keys_per_driver['화물_vector'] = train_cargo_keys_per_driver['화물키'].apply(get_cargo_vectors)

# 결과 출력
print(train_cargo_keys_per_driver)


            차주키                                                화물키  \
0             4                  [108034501, 108034511, 108154049]   
1             6        [92498686, 108071899, 108078089, 108303713]   
2            12                             [108111304, 108317978]   
3            22                  [107834629, 107967753, 108220640]   
4            45                             [108269959, 108307071]   
...         ...                                                ...   
18103     99995  [107724802, 107818206, 108232089, 108245453, 1...   
18104     99998  [107726321, 107737422, 107766090, 92272036, 92...   
18105     99999          [92498270, 92676117, 108362127, 92801578]   
18106  33431833                             [108318026, 108482388]   
18107  40217675  [108302997, 108306743, 108306749, 108306958, 1...   

                                               화물_vector  
0      [[0.13905032, 0.5223721, 0.83562344, -0.192126...  
1      [[0.30388033, 0.45143417, 1.090115

In [19]:
# 차주키별로 화물키 리스트 생성
test_cargo_keys_per_driver = test_df.groupby('차주키')['화물키'].apply(list).reset_index()

# 화물키에 해당하는 화물_vector를 찾아 리스트로 반환하는 함수
def get_cargo_vectors(cargo_keys):
    vectors = test_df[test_df['화물키'].isin(cargo_keys)]['화물_vector'].tolist()
    return vectors

# 차주키별로 화물키 리스트 생성
test_cargo_keys_per_driver = test_df.groupby('차주키')['화물키'].apply(list).reset_index()

# 차주키별로 화물키에 해당하는 화물_vector 리스트 추가
test_cargo_keys_per_driver['화물_vector'] = test_cargo_keys_per_driver['화물키'].apply(get_cargo_vectors)

# 결과 출력
print(test_cargo_keys_per_driver)

            차주키                                                화물키  \
0             4                           [119099, 957508, 957471]   
1             6                                  [970366, 1028529]   
2            12                                           [141758]   
3            22  [139338, 146412, 561963, 562222, 760384, 1018933]   
4            45                                           [520151]   
...         ...                                                ...   
15327     99995                           [702904, 833650, 968319]   
15328     99998  [169910, 213450, 471035, 494979, 518844, 54558...   
15329     99999                                   [171149, 636294]   
15330  33431833                                           [825527]   
15331  40217675   [139288, 391895, 391883, 684768, 822534, 957286]   

                                               화물_vector  
0      [[-0.6594891, 0.7715617, -0.4239521, 0.0408068...  
1      [[-0.82984644, 0.8318245, -0.56253

In [25]:
recommendations_with_similarity

,차주키,추천_화물키,코사인_유사도,순위
0,56129,107787794,0.130554,1
1,56129,107787795,0.130554,2
2,56129,108357031,0.127012,3
3,56129,92437521,0.126243,4
4,56129,107723666,0.121726,5
...,...,...,...,...
459955,89302,108485952,0.105935,26
459956,89302,108415427,0.105479,27
459957,89302,108349685,0.104835,28
459958,89302,108282904,0.104716,29


In [26]:
# 차주_vector 모델의 파라미터 수
vehicle_model_params = len(vehicle_model.wv.index_to_key) * vehicle_model.vector_size

# 화물_vector 모델의 파라미터 수
cargo_model_params = len(cargo_model.wv.index_to_key) * cargo_model.vector_size

print("차주_vector 모델의 파라미터 수:", vehicle_model_params)
print("화물_vector 모델의 파라미터 수:", cargo_model_params)


차주_vector 모델의 파라미터 수: 2396900
화물_vector 모델의 파라미터 수: 2748500


In [99]:
# df와 recommendations_with_similarity를 차주키를 기준으로 결합
merged_df = pd.merge(df, recommendations_with_similarity, on='차주키', how='inner')

# 결합된 데이터프레임의 상위 몇 개 행을 출력하여 확인
print(merged_df.head())

        화물키       화물정보        상차지        상차경도       상차위도        하차지  \
0  92218976  cj택배  셔틀   경기 광주 도척면  127.319476  37.309033  대전 대덕 문평동   
1  92218976  cj택배  셔틀   경기 광주 도척면  127.319476  37.309033  대전 대덕 문평동   
2  92218976  cj택배  셔틀   경기 광주 도척면  127.319476  37.309033  대전 대덕 문평동   
3  92218976  cj택배  셔틀   경기 광주 도척면  127.319476  37.309033  대전 대덕 문평동   
4  92218976  cj택배  셔틀   경기 광주 도척면  127.319476  37.309033  대전 대덕 문평동   

         하차경도       하차위도  차종류  차톤수  ... 화물실중량                      등록날짜  \
0  127.401436  36.446504  윙바디  11톤  ...     7 2020-04-01 00:00:00+00:00   
1  127.401436  36.446504  윙바디  11톤  ...     7 2020-04-01 00:00:00+00:00   
2  127.401436  36.446504  윙바디  11톤  ...     7 2020-04-01 00:00:00+00:00   
3  127.401436  36.446504  윙바디  11톤  ...     7 2020-04-01 00:00:00+00:00   
4  127.401436  36.446504  윙바디  11톤  ...     7 2020-04-01 00:00:00+00:00   

          요일    차주키            운행시간            잡힌시간 카테고리    추천_화물키   코사인_유사도  \
0  Wednesday  47982 0 days 06:00:00 0 days

In [100]:
merged_df['is_correct'] = merged_df['화물키'] == merged_df['추천_화물키']

In [101]:
# 정확도 계산
accuracy = merged_df['is_correct'].mean()
print(f"추천 시스템의 정확도: {accuracy:.2f}")


추천 시스템의 정확도: 0.00


In [102]:
# 각 사용자별 정밀도를 저장할 리스트
precision_at_5_list = []

# 사용자별로 반복
for user_id in merged_df['차주키'].unique():
    # 현재 사용자의 데이터 추출
    user_data = merged_df[merged_df['차주키'] == user_id]
    
    # 상위 5개의 추천 아이템 중 실제 사용자가 선호하는 아이템의 수 계산
    correct_recommendations = user_data[user_data['is_correct'] == True]
    precision_at_5 = len(correct_recommendations) / 5
    
    # 리스트에 추가
    precision_at_5_list.append(precision_at_5)

# MAP@5 계산
map_at_5 = sum(precision_at_5_list) / len(precision_at_5_list)
print(f"MAP@5: {map_at_5:.2f}")


MAP@5: 0.00


In [63]:
# 예시: 두 벡터 간의 코사인 유사도 계산 함수
def calculate_cosine_similarity(vector_a, vector_b):
    # 벡터를 2D 배열로 재구성
    vector_a = np.array(vector_a).reshape(1, -1)
    vector_b = np.array(vector_b).reshape(1, -1)
    
    # 코사인 유사도 계산
    similarity = cosine_similarity(vector_a, vector_b)
    
    return similarity[0][0]

# 추천 화물 벡터와 실제 화물 벡터 예제
recommended_cargo_vector = 
actual_cargo_vector = 

# 유사도 계산
similarity_score = calculate_cosine_similarity(recommended_cargo_vector, actual_cargo_vector)
print(f"유사도 점수: {similarity_score}")

유사도 점수: 0.9986254289035239


In [40]:
# def recommend_cargos_with_similarity(test_df, train_df, top_n=5):
#     recommendations_list = []  # 결과를 저장할 리스트
    
#     # train_df에서 화물 벡터 준비
#     cargo_vectors = np.array(list(train_df['화물_vector']))
    
#     for index, row in test_df.iterrows():
#         driver_vector = np.array(row['차주_vector']).reshape(1, -1)
        
#         # 코사인 유사도 계산
#         similarity_scores = cosine_similarity(driver_vector, cargo_vectors)[0]
        
#         # 가장 유사도가 높은 화물 인덱스 찾기 (상위 5개)
#         top_cargo_indices = np.argsort(similarity_scores)[-top_n:][::-1]
        
#         # 상위 5개 화물의 키와 코사인 유사도 추출
#         for rank, cargo_index in enumerate(top_cargo_indices):
#             recommended_cargo_key = train_df.iloc[cargo_index]['화물키']
#             similarity_score = similarity_scores[cargo_index]
            
#             # 결과 저장 (코사인 유사도 점수 포함)
#             recommendations_list.append({
#                 '차주키': row['차주키'], 
#                 '추천_화물키': recommended_cargo_key, 
#                 '코사인_유사도': similarity_score,
#                 '순위': rank + 1  # 추천 순위
#             })
    
#     # 리스트에서 DataFrame 생성
#     recommendations = pd.DataFrame(recommendations_list)
#     return recommendations

# # 추천 실행 및 결과 출력
# recommendations_with_similarity = recommend_cargos_with_similarity(test_df, train_df, top_n=5)
# print(recommendations_with_similarity)

In [43]:
df['차주키'].unique()

array([47982, 85445, 83154, ..., 88585, 87895, 81913], dtype=int64)

In [17]:
# 사용자 입력 받기
차주키_입력 = input("확인하고 싶은 차주키를 입력하세요: ")

차주키_입력 = int(차주키_입력)

# 입력 받은 차주키에 해당하는 데이터 필터링
차주_데이터 = df[df['차주키'] == 차주키_입력]

# 결과 출력
print(차주_데이터)

확인하고 싶은 차주키를 입력하세요: 56129
              화물키                         화물정보             상차지        상차경도  \
644917   92776596                   당착 상승 사료백    경기 용인 처인구 남사면  127.150493   
645920   92778334                    일양택배5축가능           경기 남양주  127.216528   
646137   92778974                     cj택배 셔틀       경기 구리 토평동   127.149034   
646569  108372314                       원지롤 당착       충남 세종 부강면  127.381046   
647305   92779925              사료파렛 통윙바디 광폭 상승       인천 중구 북성동  126.618535   
654577  108381632                          제지롤       울산 울주 온산읍  129.335467   
661902  108389605               내일착 주류빠렛 15시반전       경남 마산 구암동  128.581433   
669893  108398462           휴지 19시바로상 바로출 바로하차      충남 세종 조치원읍  127.298525   
672131   92808303             택배 인천 대전 남해  내일착       인천 중구 신흥동  126.634006   
674852  108403605                  생수12파렛2착 당착       경남 산청 삼장면  127.801327   
682848  108412955                     생수18p 당착       충북 청원 가덕면  127.561114   
684751  108416269                     

In [110]:
# 사용자 입력 받기
화물키_입력 = input("확인하고 싶은 화물키를 입력하세요: ")

화물키_입력 = int(화물키_입력)

# 입력 받은 차주키에 해당하는 데이터 필터링
화물_데이터 = df[df['화물키'] == 화물키_입력]

# 결과 출력
print(화물_데이터)

확인하고 싶은 화물키를 입력하세요: 107701282
           화물키          화물정보        상차지        상차경도       상차위도        하차지  \
584  107701282  철자재 현대제철  당착  충남 당진 송악면  126.742749  36.924837  대구 달성 구지면   

           하차경도       하차위도 차종류  차톤수  ... 직선거리                등록시간  \
584  128.405166  35.657643  카고  25톤  ...  282 2020-04-01 08:17:00   

                   배차시간  화물실중량                      등록날짜         요일    차주키  \
584 2020-04-01 08:17:00     25 2020-04-01 00:00:00+00:00  Wednesday  47749   

               운행시간   잡힌시간 카테고리  
584 0 days 05:00:00 0 days   철재  

[1 rows x 27 columns]


In [41]:
test_df

,화물키,화물정보,상차지,상차경도,상차위도,하차지,하차경도,하차위도,차종류,차톤수,...,등록날짜,요일,차주키,운행시간,잡힌시간,카테고리,차주_vector_data,화물_vector_data,차주_vector,화물_vector
758354,102114,이마트믈류,충남 천안 서북구 성거읍,127.190742,36.877034,경기 시흥,126.803102,37.379888,윙바디,11톤,...,2020-09-01 00:00:00+00:00,Tuesday,56129,0 days 03:00:00,0 days 00:23:00,택배,윙바디11톤350000인수증0 days 00:23:000 days 03:00:00,일반카고일반일반택배1192,"[-0.24648827, 0.19417068, -0.04132443, 0.11085...","[-0.4945095, -0.06372186, -0.123926565, -0.211..."
758355,121351,낼07시상차 제품파렛 10시40분하차,충북 청원 남이면,127.429134,36.573516,인천,126.705206,37.456256,카고윙바디,11톤,...,2020-09-01 00:00:00+00:00,Tuesday,85477,0 days 03:30:00,0 days 08:09:00,제품,카고윙바디11톤290000인수증0 days 08:09:000 days 03:30:00,일반카고일반일반제품9146,"[-0.2795662, 0.20445272, -0.080147795, 0.15341...","[-0.4576333, -0.07712255, -0.115949444, -0.229..."
758356,118689,쿠팡 인천 동탄 동탄 창원 16시상차,인천 서구 오류동,126.611536,37.580834,경남 창원,128.640261,35.253843,윙바디,11톤,...,2020-09-01 00:00:00+00:00,Tuesday,22569,0 days 16:00:00,0 days 05:05:00,택배,윙바디11톤500000인수증0 days 05:05:000 days 16:00:00,일반카고일반일반택배7399,"[-0.20390633, 0.20406437, -0.045287713, 0.1199...","[-0.4901595, -0.066397846, -0.12431621, -0.209..."
758357,118590,당착,대전 대덕 문평동,127.401436,36.446504,전남 광양 광양읍,127.600255,34.955892,윙바디,11톤,...,2020-09-01 00:00:00+00:00,Tuesday,84744,0 days 06:00:00,0 days 02:00:00,날짜,윙바디11톤350000인수증0 days 02:00:000 days 06:00:00,일반카고일반일반날짜8205,"[-0.20910788, 0.2518971, -0.032733228, 0.13938...","[-0.497314, -0.06584937, -0.14279796, -0.21861..."
758358,118545,새벽02시상차 한진택배,경기 군포,126.935174,37.361670,인천,126.705206,37.456256,윙바디,11톤,...,2020-09-01 00:00:00+00:00,Tuesday,21957,0 days 06:00:00,0 days 01:05:00,택배,윙바디11톤280000인수증0 days 01:05:000 days 06:00:00,일반카고일반일반택배1131,"[-0.22351706, 0.20919299, -0.059788957, 0.1065...","[-0.49241856, -0.064373635, -0.12422834, -0.21..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774349,212133,파레트 내일상하차,울산 남구 여천동,129.360986,35.525537,부산 강서 화전동,128.870855,35.104153,카고윙바디,18톤,...,2020-09-01 00:00:00+00:00,Thursday,40396,0 days 05:00:00,0 days 00:00:00,파렛트,카고윙바디18톤160000인수증0 days 00:00:000 days 05:00:00,일반카고일반일반파렛트1886,"[-0.30597487, 0.24099922, -0.002880555, 0.1675...","[-0.5082118, -0.06380281, -0.14789055, -0.2119..."
774350,212512,파레트 내일상하차,울산 남구 여천동,129.360986,35.525537,경남 김해 화목동,128.860494,35.192594,카고윙바디,18톤,...,2020-09-01 00:00:00+00:00,Thursday,50857,0 days 05:00:00,0 days 00:10:00,파렛트,카고윙바디18톤160000인수증0 days 00:10:000 days 05:00:00,일반카고일반일반파렛트1881,"[-0.2815692, 0.23427078, -0.0339514, 0.1671801...","[-0.5055089, -0.06526443, -0.1458593, -0.21078..."
774351,214367,예약 냉연1코일 복포 당착,충남 당진 송악ic,126.778756,36.934525,대구,128.601445,35.871435,카고,25톤,...,2020-09-01 00:00:00+00:00,Thursday,96513,0 days 04:00:00,0 days 01:08:00,철재,카고25톤330000인수증0 days 01:08:000 days 04:00:00,일반카고일반일반철재21256,"[-0.25207216, 0.2407978, -0.029856319, 0.15542...","[-0.49224487, -0.06028217, -0.12895559, -0.189..."
774352,212157,파이프 비무관ㆍ고임목 없음,경남 마산,128.579530,35.220635,전남 영암 대불,126.424095,34.777000,카고,25톤,...,2020-09-01 00:00:00+00:00,Thursday,8591,0 days 22:00:00,0 days 00:01:00,자재,카고25톤400000인수증0 days 00:01:000 days 22:00:00,일반카고일반일반자재25242,"[-0.29965848, 0.2085565, -0.010326775, 0.12704...","[-0.48945606, -0.059481822, -0.13576028, -0.18..."


In [42]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_dcg(scores):
    """주어진 점수로 DCG를 계산합니다."""
    return np.sum((2**scores - 1) / np.log2(np.arange(2, len(scores) + 2)))

def calculate_ndcg(relevances, recommendations):
    """실제 관련성 점수와 추천된 아이템의 점수를 바탕으로 NDCG를 계산합니다."""
    dcg = calculate_dcg(recommendations)
    idcg = calculate_dcg(sorted(relevances, reverse=True))
    ndcg = dcg / idcg
    return ndcg

# 예시: test_df의 한 화물 벡터와 train_df의 모든 화물 벡터 간의 코사인 유사도를 사용하여 관련성 점수를 계산
test_vector = np.array([test_df['화물_vector'].iloc[0]])  # test_df의 첫 번째 화물 벡터
train_vectors = np.array(list(train_df['화물_vector']))  # train_df의 모든 화물 벡터

# 코사인 유사도 계산
similarities = cosine_similarity(test_vector, train_vectors)[0]

# 관련성 점수와 추천 점수가 동일한 간단한 예시 (실제 사용 시에는 다를 수 있음)
relevances = similarities
recommendations = similarities

# NDCG 계산
ndcg_score = calculate_ndcg(relevances, recommendations)
print(f"NDCG: {ndcg_score}")


TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'list'

In [84]:
train_df['차주_vector'] = train_df['차주_vector'].apply(lambda x: x[0])
train_df['화물_vector'] = train_df['화물_vector'].apply(lambda x: x[0])
test_df['차주_vector'] = test_df['차주_vector'].apply(lambda x: x[0])
test_df['화물_vector'] = test_df['화물_vector'].apply(lambda x: x[0])

In [ ]:
# # 시간 관련 데이터를 수치형으로 변환하기위해 유닉스 타임스탬프 값으로 변환

# train_df['상차시간'] = train_df['상차시간'].astype(np.int64) // 10**9
# train_df['하차시간'] = train_df['하차시간'].astype(np.int64) // 10**9
# train_df['운행시간'] = train_df['운행시간'].astype(np.int64) // 10**9
# train_df['잡힌시간'] = train_df['잡힌시간'].astype(np.int64) // 10**9

# test_df['상차시간'] = test_df['상차시간'].astype(np.int64) // 10**9
# test_df['하차시간'] = test_df['하차시간'].astype(np.int64) // 10**9
# test_df['운행시간'] = test_df['운행시간'].astype(np.int64) // 10**9
# test_df['잡힌시간'] = test_df['잡힌시간'].astype(np.int64) // 10**9


In [86]:
# from sklearn.preprocessing import StandardScaler

# train_numeric_features = train_df[['상차시간','하차시간', '기사운임','직선거리','운행시간','잡힌시간']]

# test_numeric_features = test_df[['상차시간','하차시간', '기사운임','직선거리','운행시간','잡힌시간']]

# # 수치형 데이터 표준화
# scaler = StandardScaler()
# train_numeric_features = scaler.fit_transform(train_numeric_features)
# test_numeric_features = scaler.fit_transform(test_numeric_features)

In [90]:
# final_train_df = train_df[['차주키','화물키','차주_vector','화물_vector','운행시간', '잡힌시간']]

In [ ]:
# # train_numeric_features와 test_numeric_features가 NumPy 배열인 경우, DataFrame으로 변환
# if isinstance(train_numeric_features, np.ndarray):
#     train_numeric_features = pd.DataFrame(train_numeric_features)
#     # 새로운 컬럼 이름을 지정할 수 있습니다. 예: columns=['feature1', 'feature2', ...]

# if isinstance(test_numeric_features, np.ndarray):
#     test_numeric_features = pd.DataFrame(test_numeric_features)
#     # 새로운 컬럼 이름을 지정할 수 있습니다. 예: columns=['feature1', 'feature2', ...]

# # 이제 DataFrame에 새로운 컬럼을 추가할 수 있습니다.
# train_numeric_features['차주_vector'] = train_df['차주_vector'].values
# train_numeric_features['화물_vector'] = train_df['화물_vector'].values

# test_numeric_features['차주_vector'] = test_df['차주_vector'].values
# test_numeric_features['화물_vector'] = test_df['화물_vector'].values